In [1]:
import re
import string

import pandas as pd
import numpy as np
import datetime

import transformers
import torch

from nltk import WordNetLemmatizer

import faiss

from tqdm.notebook import tqdm

# Загрузка и подготовка данных

In [3]:
data = pd.read_csv('../data/products.csv')
data = data.drop_duplicates()
tokenizer = transformers.BertTokenizer('../model/vocab.txt')
try:
    embedded_description = pd.read_csv('../data/embedded_description_no_lemmas')
    embedded_product_composition = pd.read_csv('../data/embedded_product_composition_no_lemmas')
    embedded_product_usage = pd.read_csv('../data/embedded_product_usage_no_lemmas')
    embedded_3_in_1 = pd.read_csv('../data/embedded_3_in_1')
except:
    pass

# model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel,
                                                    # transformers.DistilBertTokenizer,
                                                    # 'distilbert-base-uncased')

data.info()

C:\Users\123\AppData\Local\Temp\ipykernel_11880\2086688400.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/products.csv')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40559 entries, 0 to 40579
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   40559 non-null  int64  
 1   sku                  40559 non-null  object 
 2   name                 40559 non-null  object 
 3   brand                40559 non-null  object 
 4   brand_type           40559 non-null  object 
 5   dimension17          37838 non-null  object 
 6   dimension18          39810 non-null  object 
 7   dimension19          9360 non-null   object 
 8   dimension20          9841 non-null   object 
 9   country              34488 non-null  object 
 10  price                40559 non-null  int64  
 11  currency             0 non-null      float64
 12  old_price            40559 non-null  int64  
 13  category_type        40555 non-null  object 
 14  url                  40559 non-null  object 
 15  images               40559 non-null 

In [4]:
def text_processing(text: str) -> str:
    # оставляем пропуски без изменений
    if text is np.nan:
        return np.nan
    # приводим текст к нижнему регистру
    text = text.lower()
    # заменяем символы и знаки пунктуации
    text = re.sub('\(.*?\)', '', text)
    trans_dict = str.maketrans('', '', string.punctuation)
    text = text.translate(trans_dict)
    # избавляемся от лишних пробелов
    text = ' '.join(text.split())

    return text

In [5]:
text_columns = ['description', 'product_usage', 'product_composition']

for column in text_columns:
    data[column] = data[column].apply(text_processing)

data.head()

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyality_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
0,203730,19000039636,03,Ecooking,standard,Жидкое мыло,Унисекс,NaN,NaN,Дания,...,19000039636,203730,NaN,False,False,нежное мыло,намочите руки нанесите на них мыло очистите ру...,aqua sodium laureth sulfate cocamidopropyl bet...,organika,органика
1,229474,19000031882,Anti-stress,Botavikos,standard,Сыворотки,Женский,Увлажнение и питание,Лицо,Россия,...,19000031882,229474,NaN,False,False,• пробуждает внутреннюю энергию клеток создава...,равномерно распределите на коже когда чувствуе...,aqua niacinamide glycerin gluconolactone xanth...,organika,органика
2,229480,19000031888,Dry oil,Botavikos,standard,Масло,Женский,NaN,Лицо,Россия,...,19000031888,229480,NaN,False,False,действие,встряхните перед использованием и распылите ма...,capryliccapric triglyceride olea europaea frui...,organika,органика
3,200485,19000046442,Catnip Chaser,Petstages,standard,игрушка для животных,NaN,NaN,NaN,США,...,19000046442,200485,NaN,False,False,игрушка трек с пластиковым мячиком тубом кошач...,подбирайте игрушки в соответствии с весом и дв...,пластик,tovary-dlja-zhivotnyh,товары для животных
4,202556,19000025382,SALT FACIAL SCRUB ORIGINAL,Kosette,standard,Скраб,Унисекс,Очищение,Лицо,NaN,...,19000025382,202556,NaN,False,True,нежный скраб,нанесите на чистую и влажную кожу затем аккура...,glycerin sea salt water silica cocoglucoside s...,azija,азия


In [6]:
def lower(text: str) -> str:
    if text is np.nan:
        return np.nan
    return text.lower()

In [7]:
columns_to_lower = ['name', 'brand', 'dimension17', 'dimension18', 'dimension19', 'dimension20', 'country', 'category_type']

for column in columns_to_lower:
    data[column] = data[column].apply(lower)

data.head()

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyality_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
0,203730,19000039636,03,ecooking,standard,жидкое мыло,унисекс,NaN,NaN,дания,...,19000039636,203730,NaN,False,False,нежное мыло,намочите руки нанесите на них мыло очистите ру...,aqua sodium laureth sulfate cocamidopropyl bet...,organika,органика
1,229474,19000031882,anti-stress,botavikos,standard,сыворотки,женский,увлажнение и питание,лицо,россия,...,19000031882,229474,NaN,False,False,• пробуждает внутреннюю энергию клеток создава...,равномерно распределите на коже когда чувствуе...,aqua niacinamide glycerin gluconolactone xanth...,organika,органика
2,229480,19000031888,dry oil,botavikos,standard,масло,женский,NaN,лицо,россия,...,19000031888,229480,NaN,False,False,действие,встряхните перед использованием и распылите ма...,capryliccapric triglyceride olea europaea frui...,organika,органика
3,200485,19000046442,catnip chaser,petstages,standard,игрушка для животных,NaN,NaN,NaN,сша,...,19000046442,200485,NaN,False,False,игрушка трек с пластиковым мячиком тубом кошач...,подбирайте игрушки в соответствии с весом и дв...,пластик,tovary-dlja-zhivotnyh,товары для животных
4,202556,19000025382,salt facial scrub original,kosette,standard,скраб,унисекс,очищение,лицо,NaN,...,19000025382,202556,NaN,False,True,нежный скраб,нанесите на чистую и влажную кожу затем аккура...,glycerin sea salt water silica cocoglucoside s...,azija,азия


# Генерация эмбеддингов

In [8]:
def lemmatization(data):
    if data is np.nan:
        return ''
    return ' '.join([WordNetLemmatizer().lemmatize(word) for word in data.split()])

Загрузим RuBERT для генерации эмбеддингов

In [27]:
config = transformers.BertConfig.from_json_file(
    '../model/bert_config.json')
model = transformers.BertModel.from_pretrained(
    '../model/pytorch_model.bin', config=config).to('cuda:0')

Some weights of the model checkpoint at ../model/pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
batch_size = 1

for column in text_columns:
    text = data[column].fillna('')
    # lemmas = data[column].apply(lemmatization)
    vector = text.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512))
    # применим padding к векторам
    n = len(max(vector, key=len))
    # англ. вектор с отступами
    padded = np.array([i + [0]*(n - len(i)) for i in vector.values])

    # создадим маску для важных токенов
    attention_mask = np.where(padded != 0, 1, 0)

    embeddings = []
    for i in tqdm(range(padded.shape[0] // batch_size)):
        # преобразуем данные
        batch = torch.LongTensor(padded[batch_size*i : batch_size*(i+1)]).to('cuda:0')
        # преобразуем маску
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i : batch_size*(i+1)]).to('cuda:0')
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        # преобразуем элементы методом numpy() к типу numpy.array
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

    features = pd.DataFrame(np.concatenate(embeddings))
    features.to_csv(f'../data/embedded_{column}_no_lemmas', index=False)


In [14]:
batch_size = 1
text = []
# заменим пропуски в полях на пустую строку, для корректной генерации токенов
for d, pu, pc in data[['description', 'product_usage', 'product_composition']].values:
    if d is np.nan:
        d = ' '
    if pu is np.nan:
        pu = ' '
    if pc is np.nan:
        pc = ' '
    text.append(d + pu + pc)

data['3_in_1'] = text
text = data['3_in_1'].fillna('')
# lemmas = data['3_in_1'].apply(lemmatization)
vector = text.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512))
# применим padding к векторам
n = len(max(vector, key=len))
# англ. вектор с отступами
padded = np.array([i + [0]*(n - len(i)) for i in vector.values])

# создадим маску для важных токенов
attention_mask = np.where(padded != 0, 1, 0)

embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
    # преобразуем данные
    batch = torch.LongTensor(padded[batch_size*i : batch_size*(i+1)]).to('cuda:0')
    # преобразуем маску
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i : batch_size*(i+1)]).to('cuda:0')
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    # преобразуем элементы методом numpy() к типу numpy.array
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

features = pd.DataFrame(np.concatenate(embeddings))

features.to_csv(f'../data/embedded_3_in_1_no_lemma', index=False)

  0%|          | 0/40559 [00:00<?, ?it/s]

In [16]:
embedded_description.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.119271,0.136464,-0.008355,0.026413,0.099878,0.022849,-0.137604,0.196135,0.130590,0.283869,...,0.015723,0.044034,-0.101652,-0.010436,-0.073420,0.046543,0.078297,-0.060893,0.071176,-0.126580
1,0.438349,-0.415116,0.852780,-0.137081,-0.502088,0.008114,-0.632642,-0.282202,-0.164272,-0.228508,...,0.223047,0.003482,-0.434830,-0.219574,-0.037204,0.345793,-0.080911,0.495437,0.500580,0.012712
2,0.065119,0.017991,0.075553,-0.028850,0.134829,0.002659,0.054221,0.103911,0.126278,0.249699,...,0.001731,0.025728,-0.057699,0.003478,0.028238,0.210666,-0.004904,0.004242,0.091198,-0.197050
3,0.018086,-0.016375,0.492378,-0.407476,-0.094135,-0.257374,-0.079157,0.205838,-0.196295,-0.033481,...,0.098162,-0.044465,-0.297261,-0.297032,-0.079317,-0.214961,0.618069,0.343269,-0.123517,-0.134644
4,0.177492,0.092044,0.084529,-0.081504,0.267774,0.140675,-0.129266,0.110915,0.396590,0.027469,...,-0.055115,0.156429,0.088761,0.140440,-0.070732,0.089746,0.000989,-0.047724,0.266879,-0.184299


In [ ]:
embedded_description = embedded_description.drop('id', axis=1)

In [ ]:
embedded_description.shape[1]

In [17]:
index = faiss.IndexFlatL2(embedded_description.shape[1])
print(index.ntotal)  # пока индекс пустой
index.add(np.ascontiguousarray(embedded_description.to_numpy().astype('float32')))
print(index.ntotal)  # теперь в нем 10 000 векторов

0
40559


In [ ]:
embedded_description.head()

In [20]:
topn = 7
D, I = index.search(np.ascontiguousarray(embedded_description.to_numpy().astype('float32')[32487].reshape((1, -1))), 10)
print(I) # индексы самых похожих векторов
print(D) # расстояния, отсортированные по убыванию

[[32487 33613 25922 33626 31827 25939 32492 25935 33555 33620]]
[[0.        0.        0.        3.8042755 3.8042755 3.8042755 5.4084244
  5.4084244 5.4084244 5.76008  ]]


In [21]:
data.loc[I[0]]

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyality_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
32487,194915,19000043238,Marie,Flame moscow,standard,Гель для душа,Унисекс,NaN,Для душа и ванны,Россия,...,19000043238,194915,NaN,False,False,Гель для душа мягко очищает и увлажняет кожу н...,Разотрите капли геля между влажными ладонями и...,"Aqua, Cocamidopropyl Betaine, Sodium Lauryl Sa...",uhod,уход
33613,196653,100384-19000039071,Fresh Linen,Wax Lyrical,standard,Диффузоры,Унисекс,NaN,NaN,Великобритания,...,19000039078,196664,NaN,False,False,Чистый и бодрящий аромат свежевыстиранного бел...,"Достаньте содержимое из коробки, освободите па...",Смесь ароматических масел,parfjumerija,парфюмерия
25922,150414,19760324673,Омега-3 35% с витамином Е,ЗдравСити,standard,NaN,Унисекс,Общего действия,NaN,Россия,...,19760324673,150414,NaN,True,False,Биологически активная добавка к пище - дополни...,"детям старше 7 лет и взрослым, в том числе жен...","жир океанических рыб, оболочка (желатин, глице...",zdorov-e-i-apteka,здоровье и аптека
33626,92027,9410600015,Squeeze Therapy Shea Butter,Frudia,standard,Крем,Унисекс,Против признаков старения,Руки,Южная Корея,...,9410600015,92027,NaN,False,False,NaN,NaN,NaN,uhod,уход
31827,206804,19000059324,Undea des Iles,ORENS PARFUMS,standard,Наборы миниатюр,Унисекс,NaN,NaN,Франция,...,19000059324,206804,NaN,False,False,"Ориентальный древесный аромат, который словно ...",Нанести на пульсирующие точки,NaN,parfjumerija,парфюмерия
25939,142496,19760311598,ANTI-AGE,Эвалар,standard,NaN,Унисекс,Общего действия,NaN,Россия,...,19760311598,142496,NaN,True,False,Кальций хелат,Взрослым и детям старше 14 лет 2-3 таблетки в ...,кальция аминокислотный хелат (кальция бисглици...,zdorov-e-i-apteka,здоровье и аптека
32492,220163,19000072513,Black coconut,HERE AND NOW,standard,Ароматические свечи,Унисекс,NaN,NaN,Россия,...,19000072513,220163,NaN,False,False,NaN,При использовании свечи всегда необходимо дожи...,"100% соевый воск, ароматическое масло, хлопков...",dlja-doma,для дома
25935,154188,11167-58273000002,BROW ASSIST,Bodyography,standard,Карандаш для бровей,Женский,NaN,NaN,США,...,58273000002,131263,NaN,True,True,Усовершенствуйте форму бровей и сделайте их вы...,"используйте угловой карандаш, чтобы заполнить ...","Ozokerite, Caprylic/Capric Triglyceride, Ricin...",makijazh,макияж
33555,179934,19000023404,Christmas tree,BAGO home,standard,Диффузоры,Унисекс,NaN,NaN,Россия,...,19000023404,179934,NaN,True,False,Необыкновенный аромат хвои в сочетании с пряно...,"Достаньте содержимое из коробки, освободите фи...","Раствор растительного происхождения, смесь аро...",dlja-muzhchin,для мужчин
33620,185253,100493-19000031281,Ninfea,Millefiori Milano,standard,Диффузоры,Унисекс,NaN,NaN,Италия,...,19000031281,185252,NaN,False,False,"Ноты цветка кувшинки, фрезии и ландыша в сочет...","Откройте упаковку, достаньте бутылочку с жидко...","стекло, ароматическая композиция, ротанговые п...",parfjumerija,парфюмерия


In [38]:
embedded_description['id'] = data['id']

In [14]:
embedded_description = embedded_description.drop('id', axis=1)

In [12]:
embedded_description.shape[1]

769

In [43]:
index = faiss.IndexFlatL2(embedded_description.shape[1])
print(index.ntotal)  # пока индекс пустой
index.add(np.ascontiguousarray(embedded_description.to_numpy().astype('float32')))
print(index.ntotal)  # теперь в нем 10 000 векторов

0
40559


In [46]:
type(index)

faiss.swigfaiss_avx2.IndexFlatL2

In [45]:
faiss.write_index(index, 'faiss_index.index')

In [94]:
embedded_description.head()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,id
0,0.119182,0.136407,-0.008398,0.026280,0.099649,0.022758,-0.137390,0.196270,0.130492,0.283897,...,0.043792,-0.101606,-0.010544,-0.073203,0.046466,0.078466,-0.060975,0.071087,-0.126535,203730.0
1,0.437772,-0.414763,0.854249,-0.136624,-0.502700,0.007649,-0.632327,-0.282504,-0.164398,-0.228232,...,0.003711,-0.435499,-0.219542,-0.036994,0.346355,-0.081222,0.495510,0.500134,0.013172,229474.0
2,0.064964,0.017981,0.075442,-0.028719,0.134861,0.002396,0.054132,0.103898,0.126308,0.249536,...,0.025518,-0.057730,0.003447,0.028165,0.210503,-0.004843,0.004023,0.091254,-0.196976,229480.0
3,0.017483,-0.016258,0.491108,-0.407509,-0.093863,-0.257953,-0.079588,0.206367,-0.196093,-0.034315,...,-0.044944,-0.297170,-0.297158,-0.078902,-0.215319,0.618713,0.344027,-0.123039,-0.134758,200485.0
4,0.177582,0.092091,0.084957,-0.081574,0.267902,0.141011,-0.129121,0.111072,0.396916,0.027279,...,0.156531,0.089343,0.140883,-0.070492,0.089348,0.001155,-0.047962,0.267154,-0.184760,202556.0


In [92]:
topn = 7
D, I = index.search(np.ascontiguousarray(embedded_description.to_numpy().astype('float32')[32487].reshape((1, -1))), 10)
print(I) # индексы самых похожих векторов
print(D) # расстояния, отсортированные по убыванию

[[32487 26255 38538 19960 39077 31806 17582 19606 22292 39089]]
[[  0.        57.97046   86.783325 101.52981  164.57614  176.87158
  193.60435  220.60435  221.52127  247.15674 ]]


In [93]:
data.loc[I[0]]

,id,sku,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,...,main_product_sku,main_product_id,best_loyality_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
32487,194915,19000043238,Marie,Flame moscow,standard,Гель для душа,Унисекс,NaN,Для душа и ванны,Россия,...,19000043238,194915,NaN,False,False,Гель для душа мягко очищает и увлажняет кожу н...,Разотрите капли геля между влажными ладонями и...,"Aqua, Cocamidopropyl Betaine, Sodium Lauryl Sa...",uhod,уход
26255,194916,19000043239,Sonia,Flame moscow,standard,Гель для душа,Унисекс,NaN,Для душа и ванны,Россия,...,19000043239,194916,NaN,False,False,Гель для душа мягко очищает и увлажняет кожу н...,Разотрите капли геля между влажными ладонями и...,"Aqua, Cocamidopropyl Betaine, Sodium Lauryl Sa...",uhod,уход
38538,194909,19000035116,Original Eau De Cologne,4711 Acqua Colonia,standard,Одеколон,Унисекс,NaN,NaN,Германия,...,19000035116,194909,NaN,False,False,4711 Original Eau de Cologne - это один из ста...,Это уникальный бренд с интересной историей ста...,NaN,parfjumerija,парфюмерия
19960,194920,19000043243,Sonia,Flame moscow,standard,Крем,Унисекс,Увлажнение и питание,Руки,Россия,...,19000043243,194920,NaN,False,False,Крем для рук и ногтей с питательными маслами ш...,Использовать только для наружного применения. ...,"Вода, кококаприлат, сорбитол, цетеариловый спи...",uhod,уход
39077,194907,19000042801,Wellaflex Вуаль диких ягод,Wella,standard,Сухой шампунь,Женский,NaN,NaN,Германия,...,19000042801,194907,NaN,False,False,Сухой шампунь,"Хорошо встряхните флакон и, удерживая его на р...","Alcohol Denat., Butane, Propane, Isobutane, Ta...",volosy,волосы
31806,194926,19000037956,All About Clean Liquid Facial Soap - Oily Skin...,Clinique,standard,Жидкое мыло,Женский,Очищение,Лицо,США,...,19000037956,194926,NaN,False,False,NaN,NaN,NaN,uhod,уход
17582,194902,19000035189,WOLFTHORN,Old Spice,standard,Дезодорант,Мужской,NaN,Тело,США,...,19000035189,194902,NaN,False,False,"Кто знал, что аромат свободы Wolfthorn может и...","Alcohol Denat., Butane, Isobutane, Propane, Me...",NaN,dlja-muzhchin,для мужчин
19606,194901,19000035188,WOLFTHORN,Old Spice,standard,Дезодорант,Мужской,NaN,Тело,США,...,19000035188,194901,NaN,False,False,"Кто знал, что аромат свободы Wolfthorn может и...","Dipropylene Glycol, Aqua, Propylene Glycol, So...",NaN,dlja-muzhchin,для мужчин
22292,194903,19000035190,WHITEWATER,Old Spice,standard,Дезодорант,Мужской,NaN,Тело,США,...,19000035190,194903,NaN,False,False,"Многие считают, что вода — самый мощный элемен...","Alcohol Denat., Butane, Isobutane, Propane, Pa...",NaN,dlja-muzhchin,для мужчин
39089,194906,19000042800,Wellaflex Нежная роза,Wella,standard,Сухой шампунь,Женский,NaN,NaN,Германия,...,19000042800,194906,NaN,False,False,Сухой шампунь,"Хорошо встряхните флакон и, удерживая его на р...","Alcohol Denat., Butane, Propane, Isobutane, Ta...",volosy,волосы


In [79]:
r = np.ascontiguousarray(np.random.random((4, 4)))

In [81]:
r

array([[0.6156839 , 0.68898451, 0.98493178, 0.87244391],
       [0.30351483, 0.63845555, 0.81861868, 0.9121769 ],
       [0.97482156, 0.67080411, 0.26230048, 0.308565  ],
       [0.58294434, 0.98529858, 0.47673406, 0.5169393 ]])

In [84]:
r[2].reshape((1, -1))

array([[0.97482156, 0.67080411, 0.26230048, 0.308565  ]])